In [1]:
import torch
from torch import utils
from torchvision import datasets, transforms

from torch.utils.data import DataLoader


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
path='./MNIST_data'

#### Смотрим на данные

In [3]:
train_data = datasets.MNIST(path, train=True, download=True)
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data
    Split: Train

In [4]:
# параметры датасета
len_train = len(train_data)
print(f"кол-во элементов в датасете: {len_train}")
print(f"тип элемента:  {type(train_data[0])}")
print(f"длинна элемента:  {len(train_data[0])}")

кол-во элементов в датасете: 60000
тип элемента:  <class 'tuple'>
длинна элемента:  2


In [5]:
# посмотрим на содержимое 
print(type(train_data.data[0]))
print(train_data.data[0].shape)



<class 'torch.Tensor'>
torch.Size([28, 28])


#### Вычисления и преобразования

In [6]:
def get_train_mean_std(train_set):
    """
    Получаем тренировочный датасет
    Возвращаем кортеж (mean, std) 
    ==============================
    """   
    # обращаемся к сырым дынным и трансформируем их 
    items = train_set.data.numpy()
    print(items.shape)
    return items.mean(), items.std()

In [7]:
# получим статистики
k_mean, k_std = get_train_mean_std(train_data)
print(f"Mean = {k_mean}\nStd = {k_std}")

(60000, 28, 28)
Mean = 33.318421449829934
Std = 78.56748998339798


#### проверяем результат

In [8]:
# создаем свой трансформ
tr = transforms.Normalize((k_mean,), (k_std,))
# подготавливаем данные - float и нормализуем
items = tr(train_data.data.float())

print(items.shape)
#print(items[0])
print()
print(f"Mean = {items.mean().round()}\nStd = {items.std()}")


torch.Size([60000, 28, 28])

Mean = -0.0
Std = 1.0


### OK!


##### Создаем свой transform - вариант 1

In [9]:
mnist_transform = transforms.Compose([
                transforms.Lambda(lambda x : np.array(x, dtype=float)),
                transforms.ToTensor(),
                transforms.Normalize((k_mean,), (k_std,)),
                ])

In [10]:
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_transform)
#train_data_mod[0][0]

проверка

In [11]:
mnist_dl = DataLoader(dataset=train_data_mod, batch_size=len(train_data_mod))
items, _ = next(iter(mnist_dl))

print(items.shape)
print(f"Mean = {items.mean().round()}\nStd = {items.std().round()}")

torch.Size([60000, 1, 28, 28])
Mean = -0.0
Std = 1.0


##### Создаем свой transform - вариант 2
тоже самое но скрываем все в классе

In [12]:
import torchvision.transforms.functional as TF

class MNIST_Transform:
    
    def __init__(self, mean, std, inplace=False):
        self.mean = mean
        self.std = std
        self.inplace = inplace

    def __call__(self, image):
        """
        Args:
            image: PIL image of size (C, H, W) to be normalized.

        Returns:
            Tensor: Normalized Tensor image.
        """
        arr = np.array(image, dtype=float)
        return TF.normalize(TF.to_tensor(arr), self.mean, self.std, self.inplace)


    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

    

In [13]:
mnist_class_transform = MNIST_Transform((k_mean,), (k_std,))

In [14]:
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_class_transform)
#train_data_mod[0][0]

проверка

In [15]:
mnist_dl = DataLoader(dataset=train_data_mod, batch_size=len(train_data_mod))
items, _ = next(iter(mnist_dl))

print(items.shape)
print(f"Mean = {items.mean().round()}\nStd = {items.std().round()}")

torch.Size([60000, 1, 28, 28])
Mean = -0.0
Std = 1.0
